In [1]:
from dotenv import load_dotenv
import chromadb
import os

load_dotenv('../.env.local')
storage_path = '../'+os.getenv('STORAGE_PATH')
print(storage_path)

../chromadb


In [2]:
from IPython.display import display, Markdown
def view_text_in_markdown(page_content):
    display(Markdown(page_content))

## Synonyms

In [1]:
from nltk.corpus import wordnet2022 as wn

def get_synonyms(word):
    synonyms = set()
    for synset in wn.synsets(word):
        for lemma in synset.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

query = "What is hazard button in the vehicle?"
expanded_query = [get_synonyms(word) for word in query.split()]
print(expanded_query)


[[], ['atomic_number_53', 'I', 'one', '1', 'live', 'comprise', 'constitute', 'unity', 'iodin', 'be', 'embody', 'equal', 'iodine', 'follow', 'i', 'exist', 'ace', 'represent', 'cost', 'personify', 'single', 'make_up'], ['take_chances', 'endangerment', 'luck', 'run_a_risk', 'jeopardize', 'peril', 'guess', 'venture', 'pretend', 'stake', 'fortune', 'chance', 'jeopardy', 'adventure', 'hazard', 'risk', 'gamble', 'take_a_chance'], ['push_button', 'clitoris', 'push', 'button', 'release', 'clit'], ['Indiana', 'inch', 'Ind.', 'inward', 'In', 'IN', 'indium', 'Hoosier_State', 'inwards', 'atomic_number_49', 'in'], [], []]


Resultado parece bem ruim.

## Vector Embeddings

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

In [4]:
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=True)

def get_vector_store(persist_directory):
        """
        Load an existing vector store from the persist directory.
        """
        if os.path.exists(persist_directory):
            vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
            print("Loaded existing vector store.")
            return vector_store
        else:
            raise FileNotFoundError("Persist directory not found. Please process documents first.")

In [6]:
# Load vector database
vector_db = get_vector_store('../db')

/tmp/ipykernel_3037/3641085465.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)


Loaded existing vector store.


In [ ]:
retriever = vector_db.as_retriever()

In [9]:
documents = retriever.invoke('How do I use my smartphone to unlock the car?')
documents

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.45it/s]


[Document(metadata={'page': 19, 'section': 'SMART PHONE AS A KEY — IF EQUIPPED '}, page_content='O WNER  D IGITAL  K EY  P AIRING\nYour vehicle may be equipped with the ability to use your smartphone as a Digital Key to lock/unlock and start the vehicle. In order to enable this feature, complete the following steps in the smartphone APP and the Uconnect system simultaneously:\nSmartphone Procedure:\n1.Enroll within your Brand Connect application using your smartphone device\n2.Once enrolled successfully, navigate to the Digital Key tile on the Brand Connect application home screen and press “Start Pairing”\n3.Follow the prompts to add the Digital Key to the smartphone wallet\nUconnect System Procedure:\n1.Within the App drawer, select “Digital Key App”\n2.Select “Add Owner Key”\n3.To pair, place smartphone on the Wireless Charging Pad\n4.“Digital Key Successfully Added” will appear on screen once pairing has completed\nNOTE:\n● In order to use Smart Phone As A Key, the smartphone must 

In [10]:
view_text_in_markdown(documents[0].page_content)

O WNER  D IGITAL  K EY  P AIRING
Your vehicle may be equipped with the ability to use your smartphone as a Digital Key to lock/unlock and start the vehicle. In order to enable this feature, complete the following steps in the smartphone APP and the Uconnect system simultaneously:
Smartphone Procedure:
1.Enroll within your Brand Connect application using your smartphone device
2.Once enrolled successfully, navigate to the Digital Key tile on the Brand Connect application home screen and press “Start Pairing”
3.Follow the prompts to add the Digital Key to the smartphone wallet
Uconnect System Procedure:
1.Within the App drawer, select “Digital Key App”
2.Select “Add Owner Key”
3.To pair, place smartphone on the Wireless Charging Pad
4.“Digital Key Successfully Added” will appear on screen once pairing has completed
NOTE:
● In order to use Smart Phone As A Key, the smartphone must meet certain hardware and software requirements. Please check with your smartphone manufacturer for regional compatibility requirements and feature availability
● Depending on your device, you may need to tap the smartphone against the door handle and NFC capable wireless charging pad  ð  page 71 to access and start the
vehicle. Smartphones without UWB (Ultra-wideband) technology will not support hands-free vehicle access and start functionality
● For additional safety, it is recommended that you carry the back-up NFC card with you at all times to cover instances such as loss of phone or phone with a drained battery
● For any service related activities, users are required to carry along and hand over physical keys to the service dealer
● NFC cards or devices could leave the vehicle authenticated under certain circumstances. Users should ensure they are aware of the vehicle state and lock the car to prevent misuse, theft or serious accidents
● If the vehicle detects a key (digital or physical) in the cabin, the doors will not lock. Please check for presence of phones or key fobs in the cabin
● Proximity functions for the digital key may be temporarily suspended if the digital key has been
present in the vicinity of the doors for an extended period of time
Key Sharing
As the owner you have the ability to share Digital Keys with iOS and Android users. To share a Digital Key complete the following steps:
Sharing A Digital Key
1.Beginning with the owner’s smartphone, navigate to device wallet and select the Digital Key
2.Locate the sharing icon to select the phone contact you wish to share the Digital Key with
3.To continue, select the type of access to be provided with the Digital Key (sharing options could include full access, unlock only, or valet mode)
4.It is recommended you share keys with an activation code for maximum security.
Once Digital Key options have been selected and sent to shared contact, they cannot be edited unless access is revoked and re-shared.
Receiving A Digital Key
1.Upon receiving a Digital Key invitation message from the owner, click the message link to proceed
2.Input activation code from vehicle owner
3.Digital Key will now be added to your wallet.NOTE: In certain instances, the vehicle systems may require additional time to respond to a shared digital key  unlock request (e.g. a newly created key approaches  the vehicle for the first time)
Key Deletion
Smartphone Procedure:
1.Navigate to smartphone device wallet
2.Tap the Digital Key and proceed to remove the key by using the menu option provided
Uconnect System Procedure:
1.Within the App drawer, select “Digital Key App”
2.Select which phone you wish to remove by tapping the delete button provided
3.The owner smartphone will show a delete authorization pop-up to confirm that it is safe to delete the key. Selecting “Approve” in the pop-up will confirm key deletion and selecting “Deny” will prevent the key from being deleted via the vehicle Uconnect system
NOTE:
Owners should delete all paired digital keys from the vehicle at the end of ownership. Deleting the connected vehicle APP or the resetting the Uconnect system to factory will not delete paired digital keys. Users are required to delete keys as per the process mentioned above or by reaching out to the call center to perform a ‘Return to New’ process which will delete all associated information from the cloud servers and delete all owner and shared digital keys

The first document is the appropriate one.

Would we have better answers if we only consider the first document as our source?

In [23]:
retriever_1 = vector_db.as_retriever(search_kwargs={"k": 1})

In [24]:
documents = retriever_1.invoke('How do I use my smartphone to unlock the car?')
documents

OllamaEmbeddings: 100%|██████████| 1/1 [00:07<00:00,  7.96s/it]


[Document(metadata={'page': 19, 'section': 'SMART PHONE AS A KEY — IF EQUIPPED '}, page_content='O WNER  D IGITAL  K EY  P AIRING\nYour vehicle may be equipped with the ability to use your smartphone as a Digital Key to lock/unlock and start the vehicle. In order to enable this feature, complete the following steps in the smartphone APP and the Uconnect system simultaneously:\nSmartphone Procedure:\n1.Enroll within your Brand Connect application using your smartphone device\n2.Once enrolled successfully, navigate to the Digital Key tile on the Brand Connect application home screen and press “Start Pairing”\n3.Follow the prompts to add the Digital Key to the smartphone wallet\nUconnect System Procedure:\n1.Within the App drawer, select “Digital Key App”\n2.Select “Add Owner Key”\n3.To pair, place smartphone on the Wireless Charging Pad\n4.“Digital Key Successfully Added” will appear on screen once pairing has completed\nNOTE:\n● In order to use Smart Phone As A Key, the smartphone must 

## Retrieval

In [15]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [16]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [17]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Your task is to generate five different versions of the given user question to retrieve relevant documents
    from a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
retrieved_docs = retriever.invoke('How do I use my smartphone to unlock the car?',) # get relevant documents

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  7.16it/s]


In [20]:
# big doc
retrieved_docs

[Document(metadata={'page': 19, 'section': 'SMART PHONE AS A KEY — IF EQUIPPED '}, page_content='O WNER  D IGITAL  K EY  P AIRING\nYour vehicle may be equipped with the ability to use your smartphone as a Digital Key to lock/unlock and start the vehicle. In order to enable this feature, complete the following steps in the smartphone APP and the Uconnect system simultaneously:\nSmartphone Procedure:\n1.Enroll within your Brand Connect application using your smartphone device\n2.Once enrolled successfully, navigate to the Digital Key tile on the Brand Connect application home screen and press “Start Pairing”\n3.Follow the prompts to add the Digital Key to the smartphone wallet\nUconnect System Procedure:\n1.Within the App drawer, select “Digital Key App”\n2.Select “Add Owner Key”\n3.To pair, place smartphone on the Wireless Charging Pad\n4.“Digital Key Successfully Added” will appear on screen once pairing has completed\nNOTE:\n● In order to use Smart Phone As A Key, the smartphone must 

The first document is the same as the simpler version of the retriever, not using the MultiQuery.

In [ ]:
len(retrieved_docs)

12

In [ ]:
view_text_in_markdown(retrieved_docs[1].page_content)

IF YOU NEED ASSISTANCE . Roadside Assistance... FCA US LLC Customer Assistance Center.. 355 FCA Canada Customer Care. Mexico.

Contents 7

355

Puerto Rico And US Virgin Islands ... Customer Assistance For The Hearing Or Speech Impaired (TDD/TTY). Service Contract . WARRANTY INFORMATION MOPAR® PARTS REPORTING SAFETY DEFECTS .... In The 50 United States And Washington, D.C. In Canada ORDERING AND ACCESSING ADDITIONAL OWNER’S INFORMATION ..... CHANGE OF OWNERSHIP OR ADDRESS. GENERAL INFORMATION.....

8 INTRODUCTION

INTRODUCTION

Dear Customer, Congratulations on the purchase of your new Ram vehicle. Be assured that it represents precision workmanship, distinctive styling, and high quality.

In [21]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
resposta = chain.invoke("How do I use my smartphone to unlock the car?")
view_text_in_markdown(resposta)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  7.64it/s]


 The vehicle's keyless entry system doesn't allow for smartphone unlocking. However, you can use the key fob or the passive entry handle on the door to unlock the vehicle (refer to the provided documentation, page 26). If Passive Entry (if equipped) is used for the tailgate, the rest of the vehicle doors will remain locked unless all doors are set to unlock on first press within Uconnect Settings.

In [ ]:
# Restrict documents retrieved to max=1
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(search_kwards={"k": 1}), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

resposta = chain.invoke("How do I use my smartphone to unlock the car?")
view_text_in_markdown(resposta)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  7.48it/s]


 To use your smartphone to unlock your car, you'll need a vehicle equipped with keyless entry and a compatible smartphone app. Here are the steps for using the app to unlock your car:

1. Download the app: If you haven't already, download the mobile app provided by your car manufacturer from the Apple App Store (iPhone) or Google Play Store (Android). Ensure that your phone meets the system requirements specified in the app description.

2. Register your car: Follow the registration process within the app to link your vehicle with your smartphone account. This usually involves entering some identifying information about your car, such as its VIN (Vehicle Identification Number), and confirming your identity through a verification process.

3. Enable Bluetooth on your phone: Make sure that Bluetooth is enabled on your smartphone to establish a connection with the car's key fob or compatible module.

4. Pair your phone with the car: Open the app and follow the instructions to pair your smartphone with the car. This may involve scanning a QR code displayed in the car, entering a unique activation code sent to your phone, or using near-field communication (NFC) to tap the phone against the car's key fob or module.

5. Unlock the car: Once your phone is paired with the car, you should be able to use it to lock and unlock the vehicle. To unlock the car, simply open the app and press the appropriate button within the app (usually represented by a car icon). You can also set up the app to allow for automatic unlocking when your phone is near the car or when you approach the vehicle with your phone in hand.

6. Important note: It's essential to keep your smartphone charged and the app updated to ensure that you can continue to use the keyless entry feature smoothly. Additionally, using your phone to unlock the car may not always be a substitute for having the physical key fob on hand; in some cases, it is still necessary to have the key fob to start the engine or access certain features of the vehicle. Always consult your car's owner's manual for specific instructions regarding the use of the smartphone app with your car model.

Limiting the documents retrieved generated better results. I also want to isolate the system from the MultiQueryRetriever. Since this document is very technical, I'm not sure if the LLM can generate good alternative queries. Anyways, the first document retrieved is the correct one.

In [ ]:
retriever_1 = vector_db.as_retriever(search_kwargs={"k": 1})

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever_1, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

resposta = chain.invoke("How do I use my smartphone to unlock the car?")
view_text_in_markdown(resposta)

OllamaEmbeddings: 100%|██████████| 1/1 [00:07<00:00,  7.81s/it]


 To use your smartphone as a key to unlock your car, follow these steps:

1. Enroll within your Brand Connect application using your smartphone device.
2. Navigate to the Digital Key tile on the Brand Connect application home screen and press “Start Pairing”.
3. Follow the prompts to add the Digital Key to the smartphone wallet.
4. Depending on your device, you may need to tap the smartphone against the door handle or place it on the Wireless Charging Pad to access and start the vehicle. Smartphones without UWB (Ultra-wideband) technology will not support hands-free vehicle access and start functionality.

This result is not as good as it can be, given the text reference, but in my opinion, it is better than the version with the MultiQueryRetriever.